# Multiple Regression

The goal of this notebook is to explore multiple regression and feature engineering.

You will use data on house sales in King County to predict prices using multiple regression.

In [1]:
import pandas as pd

sales = pd.read_csv('kc_house_data.csv')
sales.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Split data into training and testing.

In [2]:
import numpy as np

train_mask = np.random.rand(len(sales)) < 0.8

train_data = sales[train_mask]
test_data = sales[~train_mask]

# Learning a multiple regression model

We can use the following code to learn a multiple regression model predicting *price* basecd on the following features:

**example features = ['sqft_living', 'bedrooms', 'bathrooms']**

In [3]:
from sklearn import linear_model

example_features = ['sqft_living', 'bedrooms', 'bathrooms']

example_model = linear_model.LinearRegression()
example_model.fit(train_data[example_features], train_data['price'])

(example_model.intercept_, example_model.coef_)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


(72340.96368603007, array([   311.23574372, -58720.85104118,   8576.59924828]))

# Marking Predictions

Recall that once a model is built we can use the `.predict()` function to find the predicted values for data we pass. For example using the example model above:

In [4]:
example_predictions = example_model.predict(train_data[example_features])
example_predictions[0]

272013.18740101694

# RSS

The multivariate residual sum of squares formula looks as follows:

$$RSS(\textbf{w}) = \sum_{i=1}^{N} (y_i - \sum_{j=1}^{D} w_jh(x_i))^2 $$

where N is the number of samples in our training set, D is the number of features, h's are the feature extraction operators, that extract the features from x, and w's are the feature coefficients that weight the relative contributions of each feature extracted from the input.

*Side Note: Though this formula does not explicitly account for an intercept, we can always define a feature extraction operation that returns a constant of 1. This is semantically equivalent of having an intercept (or a feature that is invariant to the training input x)*

$$h'(x) \equiv 1$$

We can rewrite the above RSS formula in terms of matrix math. To do this, we will define a feature matrix H as follows:

$$
H =
\begin{bmatrix}
\dots & h^T(x_1) & \dots \\
\vdots & \vdots & \vdots \\
\dots & h^T(x_N) & \dots \\
\end{bmatrix}
$$

where the feature functions h return a 1xD vector containing all the features extracted from the input x. *H is a NxD matrix.*

$$
H\textbf{w} =
\begin{bmatrix}
h^T(x_1)\textbf{w}\\
\vdots\\
h^T(x_N)\textbf{w}\\
\end{bmatrix}
=
\begin{bmatrix}
\hat{y_1}\\
\vdots\\
\hat{y_N}\\
\end{bmatrix}
$$

where the y-hats are the predicted values of the N samples.

$$Residuals = (\textbf{y} - H\textbf{w})$$
$$RSS(\textbf{w}) = (\textbf{y} - H\textbf{w})^T(\textbf{y} - H\textbf{w})$$

The goal is to find the feature coefficients **w** that minimize the residual sum of squares.

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model.

In [17]:
def get_residual_sum_of_squares(model, features, outcomes):
    predictions = model.predict(features)
    residuals = outcomes - predictions
    return sum(residuals * residuals)

In [6]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])

rss_example_train

1159988687039463.8

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms), we can also consider transformations o existing features (e.g. the log of the sqft) or even "interaction" features (e.g. the product of bedrooms and bathrooms).

In [12]:
from math import log

train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x * x)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x * x)

train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(log)
test_data['log_sqft_living'] = test_data['sqft_living'].apply(log)

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

train_data.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A 

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340,5650,9,3.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,1991,98125,47.7210,-122.319,1690,7639,9,6.75,7.851661,-74.5980
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,98028,47.7379,-122.233,2720,8062,4,2.00,6.646391,-74.4951
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360,5000,16,12.00,7.580700,-74.8722
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,98074,47.6168,-122.045,1800,7503,9,6.00,7.426549,-74.4282


In [15]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

model_1 = linear_model.LinearRegression()
model_1.fit(train_data[model_1_features], train_data['price'])

model_2 = linear_model.LinearRegression()
model_2.fit(train_data[model_2_features], train_data['price'])

model_3 = linear_model.LinearRegression()
model_3.fit(train_data[model_3_features], train_data['price'])

# Comparing the bathrooms features for model_1, model_2, and model_3
(model_1.coef_[2], model_2.coef_[2], model_3.coef_[3])

(16885.61147281496, -73375.7387453636, 530302.7394041021)

We can see a massive difference between the coefficients for the bathrooms feature between models 1, 2, and 3. In particular, we can see that the bathroom feature has a positive coefficient for the first model and a negative coefficient for the second model.

This shows how adding other features can affect the interpretation of existing features. The second model introduces the feature **bed_bath_rooms** which is the product of the number of bedrooms and bathrooms. Given that feature, for 2 houses with the same **bed_bath_rooms** value but different number of bathrooms, then the house with more bathrooms means it must have fewer bedrooms, which could make the house less valuable.

# Comparing multiple models

Now that we've learned three models and extracted the model weights, we want to evaluate which model is best.

First compare the RSS on training data for each model:

In [20]:
model_1_train_rss = get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data['price'])
model_2_train_rss = get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])
model_3_train_rss = get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price'])

(model_1_train_rss, model_2_train_rss, model_3_train_rss)

(965419940957024.6, 955690004393760.4, 903982112233876.0)

Now compare the models against the test data:

In [21]:
model_1_test_rss = get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])
model_2_test_rss = get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])
model_3_test_rss = get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price'])

(model_1_test_rss, model_2_test_rss, model_3_test_rss)

(227549715888695.2, 225554197053411.7, 213260902009756.3)